# Final Project 

Author: Jiaying Wu, Chen Li, Zhao Chen

## 1. Dataset Description
The 3 dataset are from yelp open dataset (in Json file) https://www.yelp.com/dataset/documentation/main
1. business.json: Contains business data including location data, attributes, and categories.
2. review.json: Contains full review text data including the user_id that wrote the review and the business_id the review is written for.
3. user.json: User data including the user's friend mapping and all the metadata associated with the user.


In [369]:
# Load the Python libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load Surprise libraries
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy

# Load plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

import gc
import re

#map
import folium 

#preprocessing 
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
  

## 2. Data ETL
### 2.1 Business Data
The objective of this section is to choose restaurants which are open. And we changed the name of the columns to avoid repition in other 2 files. In this part, we deleted attributes and postal code. 

In [370]:
business = pd.read_json('/Users/chenzhaograce/MAS651/Final_Project/yelp_dataset/yelp_academic_dataset_business.json',lines=True)

#finding restaurants restaurants
business['restaurant']= (business.categories.str.contains(pat='Restaurant',
                                                          flags=re.IGNORECASE,
                                                          regex=True))
business.head(5)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,restaurant
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",True
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ...",True
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0...",False
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None,False
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'...",False


In [371]:
required_columns = ['business_id','name','latitude','longitude','city','state','stars','categories','review_count']  # columns to use 

restaurants = business[(business.restaurant==1)&(business.is_open==1)].reset_index(drop=True)

#keeping required cols 
restaurants=restaurants.loc[:,required_columns]

restaurant_list = restaurants.business_id.unique()   # list of restaurants ids 

print(f'{len(restaurants)} Restaurants(That are Open) in the Dataset')

32049 Restaurants(That are Open) in the Dataset


In [372]:
restaurants=restaurants.rename(columns={'stars': 'restaurant_stars','review_count': 'restaurant_review_count'})
restaurants.head(5)

,business_id,name,latitude,longitude,city,state,restaurant_stars,categories,restaurant_review_count
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,40.017544,-105.283348,Boulder,CO,4.0,"Gastropubs, Food, Beer Gardens, Restaurants, B...",86
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,45.588906,-122.593331,Portland,OR,4.0,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",126
2,D4JtQNTI4X3KcbzacDJsMw,Bob Likes Thai Food,49.251342,-123.101333,Vancouver,BC,3.5,"Restaurants, Thai",169
3,jFYIsSb7r1QeESVUnXPHBw,Boxwood Biscuit,39.947007,-82.997471,Columbus,OH,4.5,"Breakfast & Brunch, Restaurants",11
4,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,42.541155,-70.973438,Peabody,MA,4.0,"Food, Pizza, Restaurants",39


In [373]:
restaurants.to_csv('restaurants.csv',index=False)

### 2.2 Review_data
The objective in this part is to join data with business data. Since the data is really large, we choose the year during Pandemic time (after 2020-01-01). And we select random 10000 data from the dataset.

In [374]:
#taking these columns for reviews 
required_columns2 = ['review_id','user_id','business_id','text','useful','stars','funny','cool','date']



review_dict = {'id_cols': ['user_id','business_id'],
               'review_cols': ['stars','useful','funny','cool']
              }

In [375]:
reviews = pd.read_json('/Users/chenzhaograce/MAS651/Final_Project/yelp_dataset/yelp_academic_dataset_review.json',
                      lines=True,
                      orient='columns',
                      chunksize=100000)

In [376]:
restaurant_reviews = pd.DataFrame()
review_count = 0 
total_reviews =0

# as we are only concerned with restaurants , we will take only those reviews 
for chunk_df in reviews:
    
    total_reviews += chunk_df.shape[0]
    
    #take only those rows which business ids are present in the restaurants df 
    restaurant = chunk_df[chunk_df['business_id'].isin(restaurant_list)].reset_index(drop=True)
    review_count += len(restaurant)    
    
    #append the columns to DF  
    restaurant_reviews=restaurant_reviews.append(restaurant[required_columns2])
    

print(f'There are total {total_reviews} reviews (including restaurant reviews)')
print(f'There are total {review_count} Restaurant reviews in the dataset')
    
assert review_count == restaurant_reviews.shape[0] , 'Hello Darkness my old (only) Friend'

There are total 8635403 reviews (including restaurant reviews)
There are total 4305580 Restaurant reviews in the dataset


In [377]:
restaurant_reviews.head(10)

,review_id,user_id,business_id,text,useful,stars,funny,cool,date
0,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,I've stayed at many Marriott and Renaissance M...,1,2,1,1,2010-01-08 02:29:15
1,J4a2TuhDasjn2k3wWtHZnQ,RNm_RWkcd02Li2mKPRe7Eg,xGXzsc-hzam-VArK6eTvtw,"This place used to be a cool, chill place. Now...",2,1,0,0,2018-01-21 04:41:03
2,9vqwvFCBG3FBiHGmOHMmiA,XGkAG92TQ3MQUKGX9sLUhw,DbXHNl890xSXNiyRczLWAg,Probably one of the better breakfast sandwiche...,0,5,0,0,2017-12-02 18:16:13
3,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,I have been here twice and have had really goo...,0,4,0,0,2011-10-27 14:32:57
4,j_eh6Iw7qseluR-7KfZOMg,xpxWG7jQXZE6BcSeuIq4PQ,mw_qxZJraNu7Q6u0GkcMew,This is a five-star restaurant if ever I have ...,0,5,0,0,2016-08-12 21:59:03
5,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,Quickly stopped in for a UFC fight. I sat down...,0,4,0,0,2015-10-04 05:37:01
6,ucFOnqgaV40oQ2YNyz5ddQ,JHXQEayrDHOWGexs0dCviA,KXCXaF5qimmtKKqnPc_LQA,Great coffee and pastries. Baristas are excell...,0,1,0,0,2018-03-03 23:45:25
7,btNWW2kdJYfwpTDyzJO3Iw,DECuRZwkUw8ELQZfNGef2Q,zmZ3HkVCeZPBefJJxzdJ7A,Nothing special but good enough. I like anoth...,0,4,0,0,2012-12-04 04:29:47
8,3c4LFWiVUHPDQzAd8uxc3A,jySmPCkEkJR3cWJlkEs9cw,ZW7aI5FO_3q_vSzI4_zx-Q,Definitely 5 stars for the donuts. Our family...,5,5,2,4,2013-08-15 14:47:40
9,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,I think their rice dishes are way better than ...,0,4,0,0,2017-05-26 03:05:46


In [339]:
restaurant_reviews = restaurant_reviews[(restaurant_reviews.date>='2020-01-01')].reset_index(drop=True)
restaurant_reviews.sort_values('date', ascending=True)

,review_id,user_id,business_id,text,useful,stars,funny,cool,date
358198,_y-POrlch6GmE3vkhrl_jQ,o1UMvGJYyPlGbrPrMVVPkQ,SFqFFIA4Ks2oHfgEANoRlw,This restaurant being small plate- it's easy t...,1,5,0,0,2020-01-01 00:00:01
282277,-ehypbwjgP_miIEQTm9zVg,ttMFlp_gAhWsRJt0wUCPbQ,ogg_C0Nd6CVzUorahN3szA,"Ordered on NYE, so acknowledge that they may h...",0,1,0,0,2020-01-01 00:00:10
134405,VcLqlz8SbrY8p3A1uG1XmA,N1ZCuhN0EFevaIfn6uRTNQ,63DvXSks1tHIDajOGvwnRQ,Coffee review only. They advertise the 'best ...,0,2,0,0,2020-01-01 00:00:59
280088,Ex4_j8MREbRhwkSzF_O2CQ,73oJZmR6_MTaQjs1BbjepQ,4AggS6T6O-ivmuT1L9nGLQ,"Relaxing atmosphere, great service and wonderf...",0,4,0,0,2020-01-01 00:03:01
140958,Srm9G-l8x2ECJHVjYiH7Ng,7TVbCA7Oy3y3Jw9h8Skasg,3kUWjyRGWc_RUGDw9X3qUw,"Waited for 40 minutes on two wraps, only for t...",0,1,0,0,2020-01-01 00:03:04
...,...,...,...,...,...,...,...,...,...
194718,d8ugEtIqaDRZTb20kW_t-w,iWcquUJBSE62jR9L278n8A,MzPjBeeJGcvwyyRENZU89g,This spot is amazing. Well worth it for lunch ...,0,5,0,0,2021-01-28 15:23:52
262994,2jTT2ZWVLccVvac6Bhb3ng,Tui3A2NxRTjsbFT92zTMuQ,_PoE8V45eOpj7zBuB26Nhw,Not much to say... BEST FOOD EVER! My family a...,0,5,0,0,2021-01-28 15:25:31
100282,pU-ALbwvmKmLlOVkC-8HsQ,uNSfZyt-xUleywzaPkhk_Q,9pJAjoekyFksZ6fbpZ7GTw,The best Mediterranean spot in Austin! The foo...,0,5,0,0,2021-01-28 15:30:21
96433,dFL0G1pQlGHA5YR3foto5g,83BBaJTAt-QSA96LFoiUNA,mFbv0sujwZe2YTubuc_Spg,I really love the deep dish pizza here... I su...,0,4,0,0,2021-01-28 15:34:59


In [340]:
#Random 10000 reviews data
restaurant_reviews=restaurant_reviews.sample(n=10000,random_state=1)

In [378]:
restaurant_review_user_list = restaurant_reviews.user_id.unique()

In [379]:
#saving restaurant reviews as a csv 
restaurant_reviews.to_csv('restaurant_reviews.csv',
                          index=False)

### 2.3. User Information
In this sections , we will extract the information of users that have written reviews in the restaurant reviews section, and extract some relevant information. And we deleted other unnessary columns.

In [390]:
required_cols3 = ['user_id','review_count','yelping_since','useful','funny','cool','fans','average_stars','friend_count']

In [391]:
user_data = pd.read_json('/Users/chenzhaograce/MAS651/Final_Project/yelp_dataset/yelp_academic_dataset_user.json',
                       lines=True,
                       orient='columns',
                       chunksize=100000)


    
user_df = pd.DataFrame()
user_count = 0 
total_users =0

# as we are only concerned with restaurants , we will take only those users with restaurant reviews 
for chunk_df in user_data:    
    
    total_users += chunk_df.shape[0]
    
    #take only those rows which user ids are present in the restaurants review user ids  
    users = chunk_df[chunk_df['user_id'].isin(restaurant_review_user_list)].reset_index(drop=True)
    user_count += len(users)
    
    
    _ = to_datetime(df=users,datetime_column='yelping_since')
    users['yelping_since'] = users['yelping_since'].dt.year
    
    
    #count numof friends user has 
    users['friend_count']  = users['friends'].apply(lambda x: len(x.split(',')))
    #append the rows and columns to DF  
    user_df=user_df.append(users[required_cols3])
    


print(f'There are total {total_users} Users (including users with restaurant reviews)')
print(f'There are total {user_count} Users with Restaurant reviews in the dataset')
    
assert user_count == user_df.shape[0] , 'Knock Knock Whos there? Stupidity you stupid.'

There are total 2189457 Users (including users with restaurant reviews)
There are total 1364404 Users with Restaurant reviews in the dataset


In [392]:
user_metadata=user_df.rename(columns={'review_count': 'user_review_count','useful': 'user_useful', 
                                      'funny': 'user_funny', 'cool': 'user_cool','average_stars': 'user_average_stars'})
#user_metadata=user_metadata.iloc[:,:9]

In [393]:
user_metadata.head(5)

,user_id,user_review_count,yelping_since,user_useful,user_funny,user_cool,fans,user_average_stars,friend_count
0,q_QQ5kBBwlCcbL1s4NVK3g,1220,2005,15038,10030,11291,1357,3.85,5813
1,dIIKEfOgo0KqUfGQvGikPg,2136,2007,21272,10289,18046,1025,4.09,6296
2,D6ErcUnFALnCQN4b1W_TlA,119,2007,188,128,130,16,3.76,835
3,JnPIjvC0cmooNDfsa9BmXg,987,2009,7234,4722,4035,420,3.77,1452
4,37Hc8hr3cw0iHLoPzLK6Ow,495,2008,1577,727,1124,47,3.72,532


In [394]:
user_metadata.to_csv('user_metadata.csv',index=False)

### 2.4 Merge Data
We merge 3 dataset together and save the final data to csv file. And we uploaded the file to Github for future use.

In [481]:
df=restaurant_reviews.merge(user_metadata,on='user_id', how='inner')
yelp=df.merge(restaurants, on='business_id', how='inner')

In [482]:
yelp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4305580 entries, 0 to 4305579
Data columns (total 25 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   review_id                object        
 1   user_id                  object        
 2   business_id              object        
 3   text                     object        
 4   useful                   int64         
 5   stars                    int64         
 6   funny                    int64         
 7   cool                     int64         
 8   date                     datetime64[ns]
 9   user_review_count        int64         
 10  yelping_since            int64         
 11  user_useful              int64         
 12  user_funny               int64         
 13  user_cool                int64         
 14  fans                     int64         
 15  user_average_stars       float64       
 16  friend_count             int64         
 17  name                     ob

In [483]:
yelp

,review_id,user_id,business_id,text,useful,stars,funny,cool,date,user_review_count,...,user_average_stars,friend_count,name,latitude,longitude,city,state,restaurant_stars,categories,restaurant_review_count
0,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,I've stayed at many Marriott and Renaissance M...,1,2,1,1,2010-01-08 02:29:15,67,...,3.72,1,Renaissance Orlando at SeaWorld,28.411733,-81.468472,Orlando,FL,3.5,"Hotels, Hotels & Travel, Restaurants, Event Pl...",290
1,NOl8mfl10jRuX5vH0WnY3Q,dzPFH-A9n98R0_rEaRFkeg,0AzLzHfOJgL7ROwhdww2ew,I was a there for a professional conference. s...,0,4,0,0,2011-09-26 00:30:39,11,...,4.25,382,Renaissance Orlando at SeaWorld,28.411733,-81.468472,Orlando,FL,3.5,"Hotels, Hotels & Travel, Restaurants, Event Pl...",290
2,Qv4o4ov_sevhEuwa3OZA_A,P241NkCS4JR-QxUwpsA9Jg,0AzLzHfOJgL7ROwhdww2ew,What a nice place for our corporate conference...,1,5,1,1,2019-03-02 15:54:35,303,...,4.36,139,Renaissance Orlando at SeaWorld,28.411733,-81.468472,Orlando,FL,3.5,"Hotels, Hotels & Travel, Restaurants, Event Pl...",290
3,xxmUdt1kwgZ21N-RzCMfDg,1hJJXtCULGN8NT2dQtYbMg,0AzLzHfOJgL7ROwhdww2ew,A cavernous and well appointed branch which is...,0,4,0,0,2011-06-14 16:40:58,459,...,3.58,389,Renaissance Orlando at SeaWorld,28.411733,-81.468472,Orlando,FL,3.5,"Hotels, Hotels & Travel, Restaurants, Event Pl...",290
4,kfuK8FWw88fkQCcEa_H0Hw,zNyqbUCtGZOH6iDv_TsPFw,0AzLzHfOJgL7ROwhdww2ew,Food and service is A-ok. Server was nice bu...,0,3,0,0,2017-08-26 00:45:54,279,...,3.89,22,Renaissance Orlando at SeaWorld,28.411733,-81.468472,Orlando,FL,3.5,"Hotels, Hotels & Travel, Restaurants, Event Pl...",290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4305575,_2Sp43ly9KXmk0YEjCMbew,PfuwVsIeXBmt8-xLjvwFcA,BgEaIonKZO1pG6_rxqcy0w,"Hired them for my baby shower last weekend, an...",2,5,0,0,2016-07-03 03:23:08,15,...,3.94,1,Mendoza & Son's Catering,45.606316,-122.513093,Vancouver,WA,5.0,"Event Planning & Services, Caterers, Food Stan...",8
4305576,2n-YNeBVYZleQsDdrJ8nTw,OMwl8un_kLM3HsAhFzBkMQ,BgEaIonKZO1pG6_rxqcy0w,Hands down the best food ever. If you need a r...,1,5,0,1,2014-11-06 16:38:59,4,...,5.00,1,Mendoza & Son's Catering,45.606316,-122.513093,Vancouver,WA,5.0,"Event Planning & Services, Caterers, Food Stan...",8
4305577,A7mjoQD_YgFtSvKuawUSMw,NY0f24S6Ca3ZZMK2y8YLWw,BgEaIonKZO1pG6_rxqcy0w,I was a little worried when I hired them for a...,2,5,0,0,2011-08-15 22:43:09,10,...,4.09,1,Mendoza & Son's Catering,45.606316,-122.513093,Vancouver,WA,5.0,"Event Planning & Services, Caterers, Food Stan...",8
4305578,yiUKLTuD209a3Vvsi5VcUQ,TKpEGZZAUc5GXuYkw1SNSw,BgEaIonKZO1pG6_rxqcy0w,"The food was great, not heavy and the differen...",0,5,0,0,2011-04-20 13:53:53,5,...,1.80,1,Mendoza & Son's Catering,45.606316,-122.513093,Vancouver,WA,5.0,"Event Planning & Services, Caterers, Food Stan...",8


In [349]:
yelp.to_csv('yelp_restaurant_review.csv',index=False)